### Callback API

- 모델이 학습 중에 충돌이 발생하거나 네트워크가 끊기면, 모든 훈련 시간이 낭비될 수 있고,  
  과적합을 방지하기 위해 훈련을 중간에 중지해야 할 수도 있다.
- 모델이 학습을 시작하면 학습이 완료될 때까지 아무런 제어를 하지 못하게 되고,  
  신경망 훈련을 완료하는 데에는 몇 시간 또는 며칠이 걸릴 수 있기 때문에 모델을 모니터링하고 제어할 수 있는 기능이 필요하다.
- 훈련 시(`fit()`) `Callback API`를 등록시키면 반복 내에서 특정 이벤트 발생마다 등록된 `callback`이 호출되어 수행된다.

**ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weight_only=False, mode='auto')**

- 특정 조건에 따라 모델 또는 가중치를 파일로 저장한다.
- filepath: "weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.hdf5"와 같이 모델의 체크포인트를 저장한다.
- monitor: 모니터링할 성능 지표를 작성한다.
- save_best_only: 가장 좋은 성능을 나타내는 모델을 저장할지에 대한 여부
- save_weights_only: weights만 저장할지에 대한 여부
- mode: {auto, min, max} 중 한 가지를 작성한다. monitor의 성능 지표에 따라 좋은 경우를 선택한다.  
  *monitor의 성능 지표가 감소해야 좋은 경우 min, 증가해야 좋은 경우 max, monitor의 이름으로부터 자동으로 유추하고 싶다면 auto

**ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto',min_lr=0)**

- 특정 반복동안 성능이 개선되지 않을 때, 학습률을 동적으로 감소시킨다.
- monitor: 모니터링할 성능 지표를 작성한다.
- factor: 학습률을 감소시킬 비유, 새로운 학습률 = 기존 학습률 * factor
- patience: 학습률을 줄이기 전에 monitor할 반복 횟수
- mode: {auto, min, max} 중 한 가지를 작성한다. monitor의 성능 지표에 따라 좋은 경우를 선택한다.  
  *monitor의 성능 지표가 감소해야 좋은 경우 min, 증가해야 좋은 경우 max, monitor의 이름으로부터 자동으로 유추하고 싶다면 auto

**EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')**

- 특정 반복동안 성능이 개선되지 않을 때, 학습을 조기에 중단한다.
- monitor: 모니터링할 성능 지표를 작성한다.
- patience: Early Stopping을 적용하기 전에 monitor할 반복 횟수
- mode: {auto, min, max} 중 한 가지를 작성한다. monitor의 성능 지표에 따라 좋은 경우를 선택한다.  
  *monitor의 성능 지표가 감소해야 좋은 경우 min, 증가해야 좋은 경우 max, monitor의 이름으로부터 자동으로 유추하고 싶다면 auto

In [1]:
from tensorflow.keras.layers import Layer, Input, Flatten, Dense
from tensorflow.keras.models import Model

INPUT_SIZE = 28

def create_model():
    input_tensor = Input(shape=(INPUT_SIZE, INPUT_SIZE))
    x = Flatten()(input_tensor)
    x = Dense(64, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    output = Dense(10, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=output)
    return model

In [2]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np


def get_preprocessed_data(images, targets):
    images = np.array(images / 255.0, dtype=np.float32)
    targets = np.array(targets, dtype=np.float32)

    return images, targets

def get_preprocessed_ohe(images, targets):
    images, targets = get_preprocessed_data(images, targets)
    oh_targets = to_categorical(targets)

    return images, oh_targets

def get_train_valid_test(train_images, train_targets, test_images, test_targets, validation_size=0.2, random_state=124):
    train_images, train_oh_targets = get_preprocessed_ohe(train_images, train_targets)
    test_images, test_oh_targets = get_preprocessed_ohe(test_images, test_targets)

    train_images, validation_images, train_oh_targets, validation_oh_targets = \
    train_test_split(train_images, train_oh_targets, stratify=train_oh_targets, test_size=validation_size, random_state=random_state)

    return (train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets)

In [3]:
from tensorflow.keras.datasets import fashion_mnist

(train_images, train_targets), (test_images, test_targets) = fashion_mnist.load_data()

(train_images, train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets) = \
get_train_valid_test(train_images, train_targets, test_images, test_targets)

print(train_images.shape, train_oh_targets.shape)
print(validation_images.shape, validation_oh_targets.shape)
print(test_images.shape, test_oh_targets.shape)

(48000, 28, 28) (48000, 10)
(12000, 28, 28) (12000, 10)
(10000, 28, 28) (10000, 10)


In [10]:
 !dir callback_files

 C 드라이브의 볼륨: OS
 볼륨 일련 번호: E0EA-245E

 C:\kdt_0900_yh\ai\deep_learning\c_tensorflow\callback_files 디렉터리

2024-05-28  오전 10:16    <DIR>          .
2024-05-28  오전 10:14    <DIR>          ..
2024-05-28  오전 10:16           739,600 weights.001-0.4115-0.8069.weights.h5
2024-05-28  오전 10:14           739,600 weights.001-0.5045-0.8086.weights.h5
2024-05-28  오전 10:14           739,600 weights.002-0.3598-0.8583.weights.h5
2024-05-28  오전 10:16           739,600 weights.002-0.3729-0.8567.weights.h5
2024-05-28  오전 10:14           739,600 weights.003-0.3594-0.8733.weights.h5
2024-05-28  오전 10:16           739,600 weights.003-0.3669-0.8699.weights.h5
2024-05-28  오전 10:14           739,600 weights.004-0.3356-0.8783.weights.h5
2024-05-28  오전 10:16           739,600 weights.004-0.3461-0.8791.weights.h5
2024-05-28  오전 10:16           739,600 weights.005-0.3483-0.8857.weights.h5
2024-05-28  오전 10:14           739,600 weights.006-0.3242-0.8899.weights.h5
2024-05-28  오전 10:16           739,600 weights.006-0

### ModelCheckPoint

In [18]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

# mcp_cb = ModelCheckpoint(
#     filepath="./callback_files/weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.h5",
#     monitor='val_loss',
#     # 모든 epoch의 파일을 저장하지 않고 좋은 성능이라 판단될 경우만 저장할 때 True 설정
#     save_best_only=False,
#     save_weights_only=True,
#     mode='min'
# )

mcp_cb = ModelCheckpoint(
    filepath="./callback_files/model.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.model.keras",
    monitor='val_loss',
    # 모든 epoch의 파일을 저장하지 않고 좋은 성능이라 판단될 경우만 저장할 때 True 설정
    save_best_only=False,
    save_weights_only=False,
    mode='min'
)

history = model.fit(x=train_images, y=train_oh_targets, 
                    validation_data=(validation_images, validation_oh_targets), 
                    epochs=20, batch_size=64, callbacks=[mcp_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.7399 - loss: 0.7578 - val_acc: 0.8484 - val_loss: 0.4188
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8516 - loss: 0.4171 - val_acc: 0.8585 - val_loss: 0.3966
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8684 - loss: 0.3675 - val_acc: 0.8673 - val_loss: 0.3632
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8772 - loss: 0.3346 - val_acc: 0.8777 - val_loss: 0.3338
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8859 - loss: 0.3144 - val_acc: 0.8750 - val_loss: 0.3372
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8885 - loss: 0.2995 - val_acc: 0.8758 - val_loss: 0.3304
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8950 - loss: 0.2851 - val_acc: 0.8748 - val_loss: 0.3406
Epoch 8/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8990 - loss: 0.2714 - val_acc: 0.8865 - val_loss: 0.3097
Epoch 9/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - ac

In [19]:
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step - acc: 0.8798 - loss: 0.3792


[0.37800469994544983, 0.8809999823570251]

In [17]:
model = create_model()
model.load_weights('./callback_files/weights.020-0.3362-0.9317.weights.h5')

model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step - acc: 0.8815 - loss: 0.3673


[0.3669484853744507, 0.8791000247001648]

In [21]:
from tensorflow.keras.models import load_model

model = load_model('./callback_files/model.020-0.3389-0.9292.model.keras')
model.evaluate(test_images, test_oh_targets)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 808us/step - acc: 0.8798 - loss: 0.3792


[0.37800469994544983, 0.8809999823570251]

### ReduceLROnPlateau

In [22]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

rlr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    mode='min'
)

history = model.fit(x=train_images, y=train_oh_targets, 
                    validation_data=(validation_images, validation_oh_targets), 
                    epochs=20, batch_size=64, callbacks=[rlr_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.7442 - loss: 0.7533 - val_acc: 0.8518 - val_loss: 0.4110 - learning_rate: 0.0010
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8536 - loss: 0.4116 - val_acc: 0.8553 - val_loss: 0.3982 - learning_rate: 0.0010
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8679 - loss: 0.3604 - val_acc: 0.8641 - val_loss: 0.3622 - learning_rate: 0.0010
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8782 - loss: 0.3363 - val_acc: 0.8684 - val_loss: 0.3516 - learning_rate: 0.0010
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8874 - loss: 0.3073 - val_acc: 0.8730 - val_loss: 0.3385 - learning_rate: 0.0010
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8939 - loss: 0.2908 - val_acc: 0.8748 - val_loss: 0.3374 - learning_rate: 0.0010
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8968 - loss: 0.2807 - val_acc: 0.8834 - val_loss: 0.3185 - learning_rate: 0.0010
Epoch 

### EarlyStopping

In [23]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

ely_cb = EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min'
)

history = model.fit(x=train_images, y=train_oh_targets, 
                    validation_data=(validation_images, validation_oh_targets), 
                    epochs=20, batch_size=64, callbacks=[ely_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - acc: 0.7497 - loss: 0.7252 - val_acc: 0.8397 - val_loss: 0.4318
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8538 - loss: 0.4077 - val_acc: 0.8650 - val_loss: 0.3707
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8713 - loss: 0.3552 - val_acc: 0.8646 - val_loss: 0.3772
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8776 - loss: 0.3365 - val_acc: 0.8725 - val_loss: 0.3499
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8842 - loss: 0.3120 - val_acc: 0.8777 - val_loss: 0.3383
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8914 - loss: 0.2975 - val_acc: 0.8702 - val_loss: 0.3524
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8971 - loss: 0.2756 - val_acc: 0.8808 - val_loss: 0.3245
Epoch 8/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8997 - loss: 0.2708 - val_acc: 0.8799 - val_loss: 0.3259
Epoch 9/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - ac

In [24]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint

model = create_model()
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])

mcp_cb = ModelCheckpoint(
    filepath="./callback_files/weights.{epoch:03d}-{val_loss:.4f}-{acc:.4f}.weights.h5",
    monitor='val_loss',
    # 모든 epoch의 파일을 저장하지 않고 좋은 성능이라 판단될 경우만 저장할 때 True 설정
    save_best_only=False,
    save_weights_only=True,
    mode='min'
)

rlr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=2,
    mode='min'
)

ely_cb = EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min'
)

history = model.fit(x=train_images, y=train_oh_targets, 
                    validation_data=(validation_images, validation_oh_targets), 
                    epochs=20, batch_size=64, callbacks=[mcp_cb, rlr_cb, ely_cb])

Epoch 1/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - acc: 0.7378 - loss: 0.7558 - val_acc: 0.8467 - val_loss: 0.4243 - learning_rate: 0.0010
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8535 - loss: 0.4138 - val_acc: 0.8641 - val_loss: 0.3746 - learning_rate: 0.0010
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8690 - loss: 0.3632 - val_acc: 0.8663 - val_loss: 0.3641 - learning_rate: 0.0010
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - acc: 0.8755 - loss: 0.3423 - val_acc: 0.8763 - val_loss: 0.3361 - learning_rate: 0.0010
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8885 - loss: 0.3088 - val_acc: 0.8734 - val_loss: 0.3484 - learning_rate: 0.0010
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.8884 - loss: 0.2988 - val_acc: 0.8668 - val_loss: 0.3665 - learning_rate: 0.0010
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - acc: 0.9050 - loss: 0.2626 - val_acc: 0.8842 - val_loss: 0.3109 - learning_rate: 1.0000e-04
Ep